We want to use the model and algorithm of Tutorial--Van der Pol's dynamics with spike train observations.ipynb on the data in the folder Data. This notebook was used to observe the time taken to run the different parts of the algorithm.

In [1]:
# Libraries
import torch
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import random
import sys

# .py
sys.path.append("..")
from inference import GaussMarkovLagrange
from likelihoods import PointProcess
from mappings import AffineMapping
from transition import FixedPointSparseGP, SparseGP
from kernels import RBF
from linkfunctions import Exp
from models import PointProcessGPSDEmodel, GPSDE
sys.path.append("../Data")
from Load_plot_data import load_neuron_data, roster_plot

# Load the data and modify its format

In [2]:
#Load data
ids,times=load_neuron_data('../Data/Cellline1_Date190328_Chip2135.npz')
N=np.max(ids) # Number of neurons
trLen=np.array([np.max(times)]) #Tmax must be an array to have one for each trial

In [3]:
diftime=[times[i]-times[i-1] for i in range(1,times.shape[0])]
diftime

[4.9999999999999914e-05,
 0.00010000000000000005,
 0.0,
 0.00019999999999999987,
 0.00010000000000000026,
 9.999999999999983e-05,
 0.0,
 5.000000000000013e-05,
 0.0,
 4.99999999999997e-05,
 5.000000000000013e-05,
 0.00014999999999999996,
 0.0002000000000000001,
 0.0004499999999999999,
 0.00014999999999999996,
 0.0,
 0.00014999999999999996,
 0.0,
 5.000000000000013e-05,
 0.0004499999999999999,
 5.0000000000000565e-05,
 4.99999999999997e-05,
 0.0,
 4.99999999999997e-05,
 0.0,
 0.00014999999999999996,
 5.0000000000000565e-05,
 0.0004499999999999999,
 9.99999999999994e-05,
 0.00010000000000000026,
 0.0,
 0.00010000000000000026,
 0.0,
 0.0,
 4.99999999999997e-05,
 0.00014999999999999996,
 0.00020000000000000052,
 0.00014999999999999996,
 0.0,
 0.0004999999999999996,
 5.0000000000000565e-05,
 0.00019999999999999966,
 0.0002500000000000002,
 0.00019999999999999966,
 0.00019999999999999966,
 0.0,
 0.0,
 0.00010000000000000026,
 0.0,
 5.0000000000000565e-05,
 4.99999999999997e-05,
 0.0,
 0.0001

The data from the files is of the form
( id : neurons ids,
times : times of spike )

For the model we need a list of array where array i contains the times at which neuron i had a spike

We can observe using unique and some tests that the ids are all the numbers between 0 and 1016 except 131 and 899. We suppose that they are not present beacause they didn't have a spike.

In [4]:
Yspike=[]
for i in range(N):
    Yspike.append(times[ids==i])
print(len(Yspike))

#For now there is only one trial, to change if we want to define trials
Yspike=[Yspike]

1016


# Choices for the model

In [5]:
#Choice of variables
dtgrid = 0.004 # discretisation for solving ODEs 
xDim = 2 # two latents

In [5]:
#Choice of model components
link = Exp() # exponential link function (to define nonlinearity in likelihood )
like = PointProcess(Yspike, link, trLen, dtstep=dtgrid, nLeg=100) # point process likelihood 
kern = RBF(xDim) # RBF kernel

#To initialize the Affine mapping from latent to observations
C = 2.*np.random.rand(xDim,N) * np.random.choice([-1,1],size=(xDim,N)) 
d = 0.1*np.random.randn(1,N)
outputMapping = AffineMapping(torch.tensor(C), torch.tensor(d)) # affine output mapping, used inn the forward of the model

In [6]:
#Choose the inducing points and transition function

# generater inducing point locations on a 2D grid (for sparse gp)
xmin, xmax = -2., 2.
Zs1, Zs2 = torch.meshgrid([torch.linspace(xmin,xmax,5), torch.linspace(xmin,xmax,5)])
numZ = 25
Zs = torch.cat((Zs1.reshape(-1,1),Zs2.reshape(-1,1)),dim=1)

transfunc = SparseGP(kern, Zs) # choose sparse GP as a transition function
transfunc.q_mu.data = torch.randn(transfunc.q_mu.size()).type(torch.float64) # random initialisation for inducing point posterior mean

# uncomment to instead condition on fixed points
#Zs_fx = torch.tensor([0., 0.]).view(-1,xDim)
# transition function conditioned on fixed point
#transfunc = SparseGP(kern, Zs, Zs_fx) 

d:\logiciel\miniconda\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


# Build the model

In [7]:
#Build the model

# build point procces generative model for continuous time spike-time observations
model = PointProcessGPSDEmodel(xDim, transfunc, outputMapping, like, nLeg=100)

# assemble inference algorithm
inference = GaussMarkovLagrange(xDim, trLen, learningRate=1, dtstep=dtgrid)

# create GPSDE model object (final object)
myGPSDE = GPSDE(model, inference)

# fix inducing points on chosen grid
myGPSDE.model.transfunc.Zs.requires_grad = False

# Testing how long it takes to run the algorithm

estep number of step for inference, mstepiter for learning update, niter for everything

We first try to do only one step for every part for the all dataset (niter=1,estepIter=1,mstepiter=1) => more than 130 min and it was stopped in inference update

In [28]:
myGPSDE.variationalEM(niter=1,eStepIter=1, mStepIter=1) #estep number of step for inference, mstepiter for learning update

c:\Users\fosmo\OneDrive\Documents\GitHub\gp-sde\demos\..\kernels.py:320: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:766.)
  scaled_diffs, _ = torch.solve(mu_x2_new_diffs.transpose(-1, -2), lengthscales_new_squared.unsqueeze(-3))


KeyboardInterrupt: 

We want to see on a smaller subset of data which part of the algorithm takes longer

On 3 neurons, all the times with 1 general iteration, 1 iteration for inference and 1 iteration for learning:

inference time = 313 s, learning time= 100 s ,update time =0s, Total : 13 min

Details inference :
forward= 36s, grad= 0.1 s, backward=275s, update= 1.8

Details backward : kullberk div=231 ,Euler=44

Details Kullberk : first :71, second:165, third:1.1, fourth:0

On 3 neurons, times<120 with 1 general iteration, 1 iteration for inference and 1 iteration for learning:

inference time = 28 s, learning time= 10 s ,update time =0s, Total :1  min

Details inference :
forward=4 s, grad= 0.02 s, backward=23s, update= 0

Details backward : kullberk div=18 ,Euler=5

Details Kullberk : first :6, second:11, third:0.1, fourth:0

On 100 neurons, times<120 with 1 general iteration, 1 iteration for inference and 1 iteration for learning:

inference time = 31 s, learning time= 46 s ,update time =0s, Total : 1 min 50

Details inference :
forward= 5s, grad= 0.3 s, backward=26s, update= 0

Details backward : kullberk div=20 ,Euler=6

Details Kullberk : first :7, second:12, third:0.1, fourth:0

On all neurons, times<120 with 1 general iteration, 1 iteration for inference and 1 iteration for learning:

inference time =32  s, learning time= 345 s ,update time =0s, Total : 6 min 50

Details inference :
forward=4.9 s, grad= 3 s, backward=24s, update= 0

Details backward : kullberk div= 19,Euler=5.5

Details Kullberk : first 7:, second:11, third:0.1, fourth:0

We notice that inference time doesn't depends on the number of neurons

In [9]:

N_neurons=10 
timemax=120
Y_spike_small=Yspike[0][:N_neurons] #keep only n first neuron
for i in range(N_neurons):  #reduce the time
    Y_spike_small[i]=Y_spike_small[i][Y_spike_small[i]<=timemax]
Y_spike_small=[Y_spike_small]
#print(Y_spike_small[0])
trLen_small=np.array([timemax]) #Tmax must be an array to have one for each trial


In [10]:
#Choice of model components
link = Exp() # exponential link function (to define nonlinearity in likelihood )
like = PointProcess(Y_spike_small, link, trLen_small, dtstep=dtgrid, nLeg=100) # point process likelihood 
kern = RBF(xDim) # RBF kernel

#To initialize the Affine mapping from latent to observations
C = 2.*np.random.rand(xDim,N_neurons) * np.random.choice([-1,1],size=(xDim,N_neurons)) 
d = 0.1*np.random.randn(1,N_neurons)
outputMapping = AffineMapping(torch.tensor(C), torch.tensor(d)) # affine output mapping, used inn the forward of the model

In [11]:
#Choose the inducing points and transition function

# generater inducing point locations on a 2D grid (for sparse gp)
xmin, xmax = -2., 2.
Zs1, Zs2 = torch.meshgrid([torch.linspace(xmin,xmax,5), torch.linspace(xmin,xmax,5)])
numZ = 25
Zs = torch.cat((Zs1.reshape(-1,1),Zs2.reshape(-1,1)),dim=1)

transfunc = SparseGP(kern, Zs) # choose sparse GP as a transition function
transfunc.q_mu.data = torch.randn(transfunc.q_mu.size()).type(torch.float64) # random initialisation for inducing point posterior mean

# uncomment to instead condition on fixed points
#Zs_fx = torch.tensor([0., 0.]).view(-1,xDim)
# transition function conditioned on fixed point
#transfunc = SparseGP(kern, Zs, Zs_fx) 

d:\logiciel\miniconda\lib\site-packages\torch\functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ..\aten\src\ATen\native\TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [12]:
#Build the model

# build point procces generative model for continuous time spike-time observations
model = PointProcessGPSDEmodel(xDim, transfunc, outputMapping, like, nLeg=100)

# assemble inference algorithm
inference = GaussMarkovLagrange(xDim, trLen_small, learningRate=1, dtstep=dtgrid)

# create GPSDE model object (final object)
GPSDEsmall = GPSDE(model, inference)

# fix inducing points on chosen grid
GPSDEsmall.model.transfunc.Zs.requires_grad = False

In [13]:
GPSDEsmall.variationalEM(niter=1,eStepIter=1, mStepIter=1)

timeforward5.671839714050293
time grad0.11369442939758301


c:\Users\fosmo\OneDrive\Documents\GitHub\gp-sde\demos\..\kernels.py:320: UserWarning: torch.solve is deprecated in favor of torch.linalg.solveand will be removed in a future PyTorch release.
torch.linalg.solve has its arguments reversed and does not return the LU factorization.
To get the LU factorization see torch.lu, which can be used with torch.lu_solve or torch.lu_unpack.
X = torch.solve(B, A).solution
should be replaced with
X = torch.linalg.solve(A, B) (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:766.)
  scaled_diffs, _ = torch.solve(mu_x2_new_diffs.transpose(-1, -2), lengthscales_new_squared.unsqueeze(-3))


backward time 1 : 7.613683223724365
backward time 2 : 13.402177095413208
backward time 3 : 0.1845076084136963
backward time 4 : 0.0019941329956054688
time backward kull 21.202362060546875
Time backard Euler6.508604049682617
time backward27.711963891983032
time update infer0.30518603324890137
inference time=33.80469036102295


c:\Users\fosmo\OneDrive\Documents\GitHub\gp-sde\demos\..\utils.py:140: UserWarning: torch.cholesky is deprecated in favor of torch.linalg.cholesky and will be removed in a future PyTorch release.
L = torch.cholesky(A)
should be replaced with
L = torch.linalg.cholesky(A)
and
U = torch.cholesky(A, upper=True)
should be replaced with
U = torch.linalg.cholesky(A).transpose(-2, -1).conj().
This transform will produce equivalent results for all valid (symmetric positive definite) inputs. (Triggered internally at  ..\aten\src\ATen\native\BatchLinearAlgebra.cpp:1285.)
  L = torch.cholesky(M)


learning time=12.901829481124878
update time0.0
-------------------------------------------------------
iter   objective    log-like      kl-div     f-prior
-------------------------------------------------------
   0    2238.115   -1185.200     713.672    -339.244
timeforward4.85901141166687
time grad0.04787254333496094
backward time 1 : 7.271564483642578
backward time 2 : 13.18974494934082
backward time 3 : 0.19747257232666016
backward time 4 : 0.0019965171813964844
time backward kull 20.66177535057068
Time backard Euler6.485663890838623
time backward27.1474392414093
time update infer0.32114362716674805
